# sched — Timed Event Scheduler

**Purpose**:	Generic event scheduler.

The sched module implements a generic event scheduler for running tasks at specific times. 

The scheduler class uses a time function to learn the current time, and a delay function to wait for a specific period of time. The actual units of time are not important, which makes the interface flexible enough to be used for many purposes.

The time function is called without any arguments, and should return a number representing the current time. 

The delay function is called with a single integer argument, using the same scale as the time function, and should wait that many time units before returning. By default monotonic() and sleep() from time are used, but the examples in this section use time.time(), which also meets the requirements, because it makes the output easier to understand.

To support multi-threaded applications, the delay function is called with argument 0 after each event is generated, to ensure that other threads also have a chance to run.



## Running Events With a Delay

Events can be scheduled to run after a delay, or at a specific time. To schedule them with a delay, use the enter() method, which takes four arguments.

* A number representing the delay
* A priority value
* The function to call
* A tuple of arguments for the function


In [2]:
import sched
import time

scheduler = sched.scheduler(time.time, time.sleep)

"""This example schedules two different events to run after 2 and 3 seconds respectively. 
When the event's time comes up, print_event() is called and prints the current time and the name argument passed to the event.
"""
def print_event(name, start):
    now = time.time()
    elapsed = int(now - start)
    print(f'EVENT: {time.ctime(now)} elapsed={elapsed} name={name}')


start = time.time()
print('START:', time.ctime(start))
scheduler.enter(2, 1, print_event, ('first', start))
scheduler.enter(3, 1, print_event, ('second', start))

scheduler.run()


START: Fri Feb  3 21:33:37 2023
EVENT: Fri Feb  3 21:33:39 2023 elapsed=2 name=first
EVENT: Fri Feb  3 21:33:40 2023 elapsed=3 name=second


## Canceling Events

Both enter() and enterabs() return a reference to the event that can be used to cancel it later. 

Because run() blocks, the event has to be canceled in a different thread. 

For this example, a thread is started to run the scheduler and the main processing thread is used to cancel the event.

In [4]:
import sched
import threading
import time

scheduler = sched.scheduler(time.time, time.sleep)

# Set up a global to be modified by the threads
counter = 0


def increment_counter(name):
    global counter
    print('EVENT:', time.ctime(time.time()), name)
    counter += 1
    print('NOW:', counter)


print('START:', time.ctime(time.time()))
e1 = scheduler.enter(2, 1, increment_counter, ('E1',))
e2 = scheduler.enter(3, 1, increment_counter, ('E2',))

# Start a thread to run the events
t = threading.Thread(target=scheduler.run)
t.start()

# Back in the main thread, cancel the first scheduled event.
scheduler.cancel(e1)

# Wait for the scheduler to finish running in the thread
t.join()

print('FINAL:', counter)

START: Fri Feb  3 21:40:38 2023
EVENT: Fri Feb  3 21:40:41 2023 E2
NOW: 1
FINAL: 1
